In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#IMPORTS
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
loc = 'nw2'
turbine = 'c02'
mode = 'SS2'

# GET THE DATA
package_folder = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
data_folder = os.path.join(package_folder, 'data')
models_folder = os.path.join(package_folder, 'models')
ss2_selected = pd.read_csv(os.path.join(data_folder, 'processed','nw2', turbine+'_ss2_selected_data_large.csv'))
ss2_selected['timestamp'] = pd.to_datetime(ss2_selected['timestamp'])
ss2_selected.set_index('timestamp', inplace=True)

SS1_dbscan = pd.read_parquet(os.path.join(data_folder, 'interim',loc,'tracked_modes', 'dbscan_based', loc+turbine+'_SS1_mode.parquet'))
SS2_dbscan = pd.read_parquet(os.path.join(data_folder, 'interim',loc,'tracked_modes', 'dbscan_based', loc+turbine+'_SS2_mode.parquet'))
FA1_dbscan = pd.read_parquet(os.path.join(data_folder, 'interim',loc,'tracked_modes', 'dbscan_based', loc+turbine+'_FA1_mode.parquet'))
FA2_dbscan = pd.read_parquet(os.path.join(data_folder, 'interim',loc,'tracked_modes', 'dbscan_based', loc+turbine+'_FA2_mode.parquet'))

rfe_selected_data = pd.read_parquet(os.path.join(data_folder, 'interim', loc, 'rfe_selected_data', loc+turbine+'_rfe_selected_data.parquet'))

In [4]:
# Data availability
1 - rfe_selected_data.isna().sum()/len(rfe_selected_data)

mvbc_WandelaarBuoy_10%_highest_waves        1.000000
mvbc_WandelaarBuoy_Wave_height              1.000000
mvbc_WandelaarBuoy_Sea_water_temperature    0.941740
mvbc_WandelaarMeasuringpile_Tide_TAW        0.989682
mvbc_WandelaarMeasuringpile_Air_pressure    0.999605
mean_NW2_C02_rpm                            0.973195
mean_NW2_C02_yaw                            0.973195
mean_NW2_C02_pitch                          0.973195
mean_NW2_C02_power                          0.973195
mean_NW2_C02_winddirection                  0.973195
dtype: float64

In [5]:
##Prepare the training and test data

#choose y_ to be SS2_selected but uniquely indexed keeping the index with hghest value in size column when duplicated
y_ = ss2_selected.copy()
y_ = y_.sort_values(by=['size'], ascending=False)
y_ = y_.loc[~y_.index.duplicated(keep='last')]
y_ = y_.sort_index()

#Synchronize data
Xy = pd.DataFrame(y_['mean_frequency'])
for col in rfe_selected_data.columns:
    Xy[col] = rfe_selected_data[col]
Xy.dropna(inplace=True)
y = Xy.iloc[:,0]
X_ = Xy[rfe_selected_data.columns]

#preprocess the data
from src.data.preprocessing import sin_cos_angle_inputs
from sklearn.model_selection import train_test_split

X = sin_cos_angle_inputs(X_)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# MinMaxnormalization of the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Standard linear regression
from sklearn.linear_model import LinearRegression

lin_regr = LinearRegression()
lin_regr.fit(X_train, y_train)
lin_regr_pred = pd.DataFrame(lin_regr.predict(X_test), index=y_test.index, columns=['predictions'])
print('Linear Regression', 'MSE: '+str(mean_squared_error(y_test, lin_regr_pred)), 'R2: '+str(r2_score(y_test, lin_regr_pred)))

Linear Regression MSE: 4.7186863529044845e-05 R2: 0.7647044697673796


In [7]:
# Multivariate Linear Regression
import math
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

mult_lin_regressions = {}
mult_lin_regr_pred = pd.DataFrame(index=y_test.index, columns=['predictions'])
case_lin_regr_pred = {}
caseIDs = pd.read_csv(os.path.join(data_folder, 'interim', 'nw2', 'labeled', loc+turbine+'_case.csv'))
caseIDs.set_index('timestamp', inplace=True)
caseIDs.index = pd.to_datetime(caseIDs.index, utc=True)
for case_ in caseIDs['caseID'].unique():
    try:
        case_lin_regr = LinearRegression()
        case_index = caseIDs[caseIDs['caseID']==case_].index
        case_index_train = case_index[case_index.isin(X_train.index)]
        case_index_test = case_index[case_index.isin(X_test.index)]
        case_lin_regr.fit(X_train.loc[case_index_train], y_train.loc[case_index_train])
        mult_lin_regressions[case_] = case_lin_regr
    except Exception as e1:
        try:
            if math.isnan(case_):
                case_lin_regr = LinearRegression()
                case_index = caseIDs[caseIDs['caseID'].isna()].index
                case_index_train = case_index[case_index.isin(X_train.index)]
                case_index_test = case_index[case_index.isin(X_test.index)]
                case_lin_regr.fit(X_train.loc[case_index_train], y_train.loc[case_index_train])
                mult_lin_regressions[case_] = case_lin_regr
        except Exception as e2:
            print(e1, e2)
            print('Case '+str(case_)+' failed')

mult_lin_regr_pred = pd.DataFrame(columns=['predictions'])
for case_ in caseIDs['caseID'].unique():
    try:
        case_index = caseIDs[caseIDs['caseID']==case_].index
        case_index_test = case_index[case_index.isin(X_test.index)]
        mult_lin_regr_pred = \
            pd.concat(
                [
                mult_lin_regr_pred,
                pd.DataFrame(mult_lin_regressions[case_].predict(X_test.loc[case_index_test]),
                             index=X_test.loc[case_index_test].index,
                             columns=['predictions'])
                ],
                axis=0)
    except Exception as e1:
        try:
            if math.isnan(case_):
                print(case_)
                case_index = caseIDs[caseIDs['caseID'].isna()].index
                case_index_test = case_index[case_index.isin(X_test.index)]
                mult_lin_regr_pred = \
                    pd.concat(
                        [
                        mult_lin_regr_pred,
                        pd.DataFrame(mult_lin_regressions[case_].predict(X_test.loc[case_index_test]), index=X_test.loc[case_index_test].index, columns=['predictions'])
                        ],
                        axis=0)
        except Exception as e2:
            print(e1, e2)
            print('Predicting Case '+str(case_)+' failed')
mult_lin_regr_pred.sort_index(inplace=True)
print('Multivariate Linear Regression',
      'MSE: '+str(mean_squared_error(y_test.loc[mult_lin_regr_pred.index], mult_lin_regr_pred)),
      'R2: '+str(r2_score(y_test.loc[mult_lin_regr_pred.index], mult_lin_regr_pred)))

Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required by LinearRegression. must be real number, not str
Case High wind: Turbine reducing output power at extreme wind speeds failed
nan
Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required by LinearRegression. must be real number, not str
Predicting Case Rated RPM: Turbine rotating at 10.4rpm or 10.445rpm failed
'High wind: Turbine reducing output power at extreme wind speeds' must be real number, not str
Predicting Case High wind: Turbine reducing output power at extreme wind speeds failed
Multivariate Linear Regression MSE: 4.4374831504075696e-05 R2: 0.7787265622919008


# Non-optimized models

In [8]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
rf_regr = RandomForestRegressor()
rf_regr.fit(X_train, y_train)
rf_regr_pred = pd.DataFrame(rf_regr.predict(X_test), index=y_test.index, columns=['predictions'])
print('Random Forest Regression', 'MSE: '+str(mean_squared_error(y_test, rf_regr_pred)), 'R2: '+str(r2_score(y_test, rf_regr_pred)))

Random Forest Regression MSE: 3.874079267951817e-05 R2: 0.8068204861815328


In [9]:
# XGBoost regression
from xgboost import XGBRegressor
xgb_regr = XGBRegressor()
xgb_regr.fit(X_train, y_train)
xgb_regr_pred = pd.DataFrame(xgb_regr.predict(X_test), index=y_test.index, columns=['predictions'])
print('XGBoost Regression', 'MSE: '+str(mean_squared_error(y_test, xgb_regr_pred)), 'R2: '+str(r2_score(y_test, xgb_regr_pred)))

XGBoost Regression MSE: 4.3271484614673016e-05 R2: 0.7842283602915224


In [10]:
# CatBoost regression
from catboost import CatBoostRegressor
cat_regr = CatBoostRegressor()
cat_regr.fit(X_train, y_train, verbose=False)
cat_regr_pred = pd.DataFrame(cat_regr.predict(X_test), index=y_test.index, columns=['predictions'])
print('CatBoost Regression', 'MSE: '+str(mean_squared_error(y_test, cat_regr_pred)), 'R2: '+str(r2_score(y_test, cat_regr_pred)))

CatBoost Regression MSE: 3.810013697163858e-05 R2: 0.8100150919088088


In [11]:
# Neural Network
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

target_scaler = MinMaxScaler()
target_scaler.fit(y_train.values.reshape(-1,1))
y_train_scaled = target_scaler.transform(y_train.values.reshape(-1,1))
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1,1))

# Define the early stopping criterion
early_stopping = EarlyStopping(
    monitor='val_mean_squared_error',  # Monitor the validation mean squared error
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    min_delta=1e-07,  # Minimum change to qualify as an improvement
    mode='min',  # Stop training when the quantity monitored has stopped decreasing
    verbose=1,
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

def build_model(layers_=[100]):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(units=layers_[0], activation='relu', input_shape=(len(X.columns),)))
    if len(layers_)>1:
        for layer in layers_[1:]:
            model.add(Dense(units=layer, activation='relu'))
    model.add(Dense(1, activation="linear"))
    model.compile(
        optimizer="adam", loss="mean_squared_error", metrics=["mean_squared_error"],
    )
    return model

nn_regr = build_model()
nn_regr.fit(X_train_scaled, y_train, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_pred = pd.DataFrame(nn_regr.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
print('Neural Network 1 hidden layer', 'MSE: '+str(mean_squared_error(y_test, nn_regr_pred)), 'R2: '+str(r2_score(y_test, nn_regr_pred)))






Restoring model weights from the end of the best epoch: 55.
Epoch 65: early stopping
29/29 [==============================] - 0s 2ms/step
Neural Network 1 hidden layer MSE: 5.6213134946613155e-05 R2: 0.7196953049197732


In [12]:
nn_regr = build_model()
nn_regr.fit(X_train_scaled, y_train_scaled, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_pred = pd.DataFrame(nn_regr.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn_regr_pred_unscaled = target_scaler.inverse_transform(nn_regr_pred)
print('Neural Network 1 hidden layer', 'MSE: '+str(mean_squared_error(y_test, nn_regr_pred_unscaled)), 'R2: '+str(r2_score(y_test, nn_regr_pred_unscaled)))

Restoring model weights from the end of the best epoch: 36.
Epoch 46: early stopping
29/29 [==============================] - 0s 2ms/step
Neural Network 1 hidden layer MSE: 3.917935370139665e-05 R2: 0.8046336180478044


In [13]:
nn_regr_2 = build_model([100, 100])
nn_regr_2.fit(X_train_scaled, y_train, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_2_pred = pd.DataFrame(nn_regr_2.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
print('Neural Network 2 hidden layers', 'MSE: '+str(mean_squared_error(y_test, nn_regr_2_pred)), 'R2: '+str(r2_score(y_test, nn_regr_2_pred)))

Restoring model weights from the end of the best epoch: 26.
Epoch 36: early stopping
29/29 [==============================] - 0s 3ms/step
Neural Network 2 hidden layers MSE: 8.778065210715737e-05 R2: 0.5622850612012935


In [14]:
nn_regr_2 = build_model([100, 100])
nn_regr_2.fit(X_train_scaled, y_train_scaled, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_2_pred = pd.DataFrame(nn_regr_2.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn_regr_2_pred_unscaled = target_scaler.inverse_transform(nn_regr_2_pred)
print('Neural Network 2 hidden layers', 'MSE: '+str(mean_squared_error(y_test, nn_regr_2_pred_unscaled)), 'R2: '+str(r2_score(y_test, nn_regr_2_pred_unscaled)))

Restoring model weights from the end of the best epoch: 27.
Epoch 37: early stopping
29/29 [==============================] - 0s 2ms/step
Neural Network 2 hidden layers MSE: 3.815231377743336e-05 R2: 0.8097549142180884


In [15]:
nn_regr_3 = build_model([100, 100, 100])
nn_regr_3.fit(X_train_scaled, y_train, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_3_pred = pd.DataFrame(nn_regr_3.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
print('Neural Network 3 hidden layers', 'MSE: '+str(mean_squared_error(y_test, nn_regr_3_pred)), 'R2: '+str(r2_score(y_test, nn_regr_3_pred)))

Restoring model weights from the end of the best epoch: 25.
Epoch 35: early stopping
29/29 [==============================] - 0s 3ms/step
Neural Network 3 hidden layers MSE: 7.684779445520165e-05 R2: 0.6168013470017104


In [16]:
nn_regr_3 = build_model([100, 100, 100])
nn_regr_3.fit(X_train_scaled, y_train_scaled, epochs=100, callbacks=[early_stopping], validation_split=0.1, verbose=0)
nn_regr_3_pred = pd.DataFrame(nn_regr_3.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn_regr_3_pred_unscaled = target_scaler.inverse_transform(nn_regr_3_pred)
print('Neural Network 3 hidden layers', 'MSE: '+str(mean_squared_error(y_test, nn_regr_3_pred_unscaled)), 'R2: '+str(r2_score(y_test, nn_regr_3_pred_unscaled)))

Restoring model weights from the end of the best epoch: 12.
Epoch 22: early stopping
29/29 [==============================] - 0s 2ms/step
Neural Network 3 hidden layers MSE: 3.9967026267643516e-05 R2: 0.8007059182546052


# Hyperopt

In [17]:
import time
time_limit = 2*60*60 # 2 hours in seconds
mode = 'SS2'
seed = 42

In [18]:
#Hyperparameter optimization: Bayesian optimization
#Hyperopt functions for hyperparameter optimizations
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from src.models.utils import convert_dict
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import numpy as np
XGB_optimizations = {}


start_time = time.time()

def objective_xgb(space):
    global start_time
    if (time.time() - start_time) > time_limit:
        raise Exception("Time limit exceeded")
    
    model = XGBRegressor(
                                 n_estimators = space['n_estimators'],
                                 max_depth = space['max_depth'],
                                 learning_rate = space['learning_rate'],
                                 colsample_bytree = space['colsample_bytree'],
                                 )
    score = cross_val_score(model,  X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    # We aim to minimize mse 
    return {'loss': -score, 'status': STATUS_OK }

def optimize_xgb(trial):
    space = {
        'n_estimators':hp.uniformint('n_estimators',10,1000),
        'max_depth':hp.uniformint('max_depth',2,20),
        'learning_rate':hp.uniform('learning_rate',0.001,0.5),
        'colsample_bytree': hp.uniform('colsample_bytree',0.1, 1),
    }
    best = \
        fmin(
            fn = objective_xgb,
            space = space,
            algo = tpe.suggest,
            trials = trial,
            max_evals = 100,
            rstate = np.random.RandomState(seed)
            )
    return best
trial2=Trials()
XGB_optimizations[mode] = optimize_xgb(trial2)
XGB_optimizations[mode] = convert_dict(XGB_optimizations)[mode]

100%|██████████| 100/100 [18:19<00:00, 11.00s/trial, best loss: 4.373629708261872e-05]


In [19]:
XGB_optimize = XGB_optimizations[mode]
regr_xgb_optimized = \
    XGBRegressor(
        n_estimators = XGB_optimize['n_estimators'],
        max_depth = XGB_optimize['max_depth'],
        learning_rate = XGB_optimize['learning_rate'],
        colsample_bytree = XGB_optimize['colsample_bytree'],
        )
regr_xgb_optimized.fit(X_train, y_train)

regr_xgb_optimized_pred = regr_xgb_optimized.predict(X_test)
regr_xgb_optimized_mse = mean_squared_error(y_test, regr_xgb_optimized_pred)
regr_xgb_optimized_r2 = r2_score(y_test, regr_xgb_optimized_pred)
print("XGBRegressor Optimized MSE:", regr_xgb_optimized_mse, "XGBRegressor Optimized R2:", regr_xgb_optimized_r2)

XGBRegressor Optimized MSE: 3.854262152372167e-05 XGBRegressor Optimized R2: 0.8078086592384525


In [20]:
# write optimal hyperparameters to a csv file
xgb_optimal_params = pd.DataFrame(XGB_optimizations[mode], index=['optimal_params']).T
xgb_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'xgb_optimal_params.csv'))

In [21]:
from hyperopt import STATUS_FAIL
start_time = time.time()
time_limit_reached = False
time_limit = 4*60*60
def objective_catboost(space):
    global start_time, time_limit_reached
    if (time.time() - start_time) > time_limit:
        time_limit_reached = True
        # Assign a large loss value to indicate failure
        return {'loss': float('inf'), 'status': STATUS_FAIL}

    model = CatBoostRegressor(
        iterations=int(space['iterations']),
        depth=int(space['depth']),
        learning_rate=space['learning_rate'],
        l2_leaf_reg=space['l2_leaf_reg'],
        verbose=False  # to make CatBoost quiet
    )
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': STATUS_OK}

def optimize_catboost(trial):
    space = {
        'iterations': hp.uniformint('iterations', 10, 1000),
        'depth': hp.uniformint('depth', 2, 16),
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.5),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
        'verbose': False
    }
    best = fmin(
        fn=objective_catboost,
        space=space,
        algo=tpe.suggest,
        trials=trial,
        max_evals=50,
        rstate=np.random.RandomState(seed)
    )
    # Check if the time limit flag was set to True
    if time_limit_reached:
        return best
        
    return best

# Example usage
CatBoost_optimizations = {}
trial_catboost = Trials()
CatBoost_optimizations[mode] = optimize_catboost(trial_catboost)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [4:47:53<00:00, 345.48s/trial, best loss: 4.132925680452247e-05]   


In [22]:
CatBoost_optimizations = convert_dict(CatBoost_optimizations)[mode]
print(CatBoost_optimizations)

# Creating and training the optimized CatBoostRegressor
regr_catboost_optimized = CatBoostRegressor(
    **CatBoost_optimizations,
    verbose=False
)
regr_catboost_optimized.fit(X_train, y_train)

# Prediction and evaluation
regr_catboost_optimized_pred = regr_catboost_optimized.predict(X_test)
regr_catboost_optimized_mse = mean_squared_error(y_test, regr_catboost_optimized_pred)
regr_catboost_optimized_r2 = r2_score(y_test, regr_catboost_optimized_pred)
print("CatBoostRegressor Optimized MSE:", regr_catboost_optimized_mse, "CatBoostRegressor Optimized R2:", regr_catboost_optimized_r2)

{'depth': 9, 'iterations': 434, 'l2_leaf_reg': 8.748492146885361, 'learning_rate': 0.05122891242301804}
CatBoostRegressor Optimized MSE: 3.6602165530033e-05 CatBoostRegressor Optimized R2: 0.8174846704793141


In [23]:
# write optimal hyperparameters to a csv file
cb_optimal_params = pd.DataFrame(CatBoost_optimizations, index=['optimal_params']).T
cb_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'cb_optimal_params.csv'))

In [24]:
from src.models.learning_rate import *
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from tensorflow.keras import backend as K

def create_model(hidden_layers, units, batch_normalization):
    model = Sequential()
    model.add(Dense(units[0], input_dim=X_train_scaled.shape[1], activation='relu'))
    if batch_normalization:
        model.add(BatchNormalization())

    for i in range(1, hidden_layers):
        model.add(Dense(units[i], activation='relu'))
        if batch_normalization:
            model.add(BatchNormalization())

    model.add(Dense(1, activation='linear'))  # Assuming a regression problem
    return model

time_limit = 2*60*60
start_time = time.time()
time_limit_reached = False

def objective_nn(params):
    global start_time, time_limit_reached
    if (time.time() - start_time) > time_limit:
        time_limit_reached = True
        # Assign a large loss value to indicate failure
        return {'loss': float('inf'), 'status': STATUS_FAIL}

    # rest of your code
    model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])
    # Learning rate schedulers
    # if params['lr_schedule'] == 'linear':
    #     lr_scheduler = LinearLearningRateScheduler(params['start_lr'], params['end_lr'], 100)
    # elif params['lr_schedule'] == 'sinusoidal':
    #     lr_scheduler = SinusoidalLearningRateScheduler(params['base_lr'], params['max_lr'], 100)
        
    

    kf = KFold(n_splits=5)
    validation_scores = []
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    for train_index, val_index in kf.split(X_train_scaled):
        # Split data into k-folds
        X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_kf, y_val_kf = y_train_scaled[train_index], y_train_scaled[val_index]

        # Create model using the provided parameters
        model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])

        # Optionally add learning rate scheduler
        optimizer = Adam(learning_rate=params['learning_rate'])
        model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"],)
        # Fit the model
        model.fit(X_train_kf, y_train_kf, epochs=100, validation_data=(X_val_kf, y_val_kf), callbacks=[early_stop], verbose=0)

        # Evaluate the model on the validation fold
        val_predictions = model.predict(X_val_kf)
        val_score = mean_squared_error(y_val_kf, val_predictions)
        validation_scores.append(val_score)

        # Clear the Keras session to prevent model/layer duplication
        K.clear_session()

    # Compute the average MSE over all folds
    average_score = np.mean(validation_scores)
    return {'loss': average_score, 'status': STATUS_OK}

def optimize(hidden_layers, trial):
    space = {
        'hidden_layers': hidden_layers,
        'units_1': hp.uniformint('units_1', 32, 256),
        'units_2': hp.uniformint('units_2', 32, 256) if hidden_layers > 1 else None,
        'units_3': hp.uniformint('units_3', 32, 256) if hidden_layers > 2 else None,
        'batch_normalization': hp.choice('batch_normalization', [False, True]),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 0.01),
        #'lr_schedule': hp.choice('lr_schedule', ['constant', 'linear', 'sinusoidal']),
        #'start_lr': hp.uniform('start_lr', 0.0001, 0.01),
        #'end_lr': hp.uniform('end_lr', 0.0001, 0.01),
        #'base_lr': hp.uniform('base_lr', 0.0001, 0.01),
        #'max_lr': hp.uniform('max_lr', 0.0001, 0.01),
    }
    best = fmin(
        fn=objective_nn,
        space=space,
        algo=tpe.suggest,
        trials=trial,
        max_evals=50,
        rstate=np.random.RandomState(seed)
    )
    # Check if the time limit flag was set to True
    if time_limit_reached:
        return best
        
    return best

In [25]:
trial_nn1 = Trials()
time_limit_reached = False
best_hyperparams_1_layer = {}

best_hyperparams_1_layer[mode] = optimize(1, trial_nn1)
best_hyperparams_1_layer = convert_dict(best_hyperparams_1_layer)[mode]
print("Best Hyperparameters for 1 layer:", best_hyperparams_1_layer)

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 2ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 2ms/step         

In [26]:
from keras.callbacks import LearningRateScheduler
# Train nn1_optimized with optimized hyperparameters

nn1_optimized = create_model(1, [best_hyperparams_1_layer['units_1']], best_hyperparams_1_layer['batch_normalization'])
nn1_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_1_layer['learning_rate']))
early_stop = EarlyStopping(monitor='val_loss', patience=5)
nn1_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# if best_hyperparams_1_layer['lr_schedule'] == 0:#'constant':
#     nn1_optimized = create_model(1, [best_hyperparams_1_layer['units_1']], best_hyperparams_1_layer['batch_normalization'])
#     nn1_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_1_layer['learning_rate']))
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn1_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# elif best_hyperparams_1_layer['lr_schedule'] == 1:#'linear':
#     nn1_optimized = create_model(1, [best_hyperparams_1_layer['units_1']], best_hyperparams_1_layer['batch_normalization'])
#     lr_scheduler = LinearLearningRateScheduler(best_hyperparams_1_layer['start_lr'], best_hyperparams_1_layer['end_lr'], 100)
#     nn1_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn1_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

# elif best_hyperparams_1_layer['lr_schedule'] == 2:#'sinusoidal':
#     nn1_optimized = create_model(1, [best_hyperparams_1_layer['units_1']], best_hyperparams_1_layer['batch_normalization'])
#     lr_scheduler = SinusoidalLearningRateScheduler(best_hyperparams_1_layer['base_lr'], best_hyperparams_1_layer['max_lr'], 100)
#     nn1_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn1_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

nn1_optimized_pred = pd.DataFrame(nn1_optimized.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn1_optimized_pred_unscaled = target_scaler.inverse_transform(nn1_optimized_pred)
nn1_optimized_mse = mean_squared_error(y_test, nn1_optimized_pred_unscaled)
nn1_optimized_r2 = r2_score(y_test, nn1_optimized_pred_unscaled)
print("Neural Network 1 hidden layer Optimized MSE:", nn1_optimized_mse, "Neural Network 1 hidden layer Optimized R2:", nn1_optimized_r2)

29/29 [==============================] - 0s 1ms/step
Neural Network 1 hidden layer Optimized MSE: 4.258648920984564e-05 Neural Network 1 hidden layer Optimized R2: 0.7876440642593557


In [27]:
from src.models.learning_rate import *
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from tensorflow.keras import backend as K

def create_model(hidden_layers, units, batch_normalization):
    model = Sequential()
    model.add(Dense(units[0], input_dim=X_train_scaled.shape[1], activation='relu'))
    if batch_normalization:
        model.add(BatchNormalization())

    for i in range(1, hidden_layers):
        model.add(Dense(units[i], activation='relu'))
        if batch_normalization:
            model.add(BatchNormalization())

    model.add(Dense(1, activation='linear'))  # Assuming a regression problem
    return model

time_limit = 2*60*60
start_time = time.time()
time_limit_reached = False

def objective_nn(params):
    global start_time, time_limit_reached
    if (time.time() - start_time) > time_limit:
        time_limit_reached = True
        # Assign a large loss value to indicate failure
        return {'loss': float('inf'), 'status': STATUS_FAIL}

    # rest of your code
    model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])
    # Learning rate schedulers
    # if params['lr_schedule'] == 'linear':
    #     lr_scheduler = LinearLearningRateScheduler(params['start_lr'], params['end_lr'], 100)
    # elif params['lr_schedule'] == 'sinusoidal':
    #     lr_scheduler = SinusoidalLearningRateScheduler(params['base_lr'], params['max_lr'], 100)
        
    

    kf = KFold(n_splits=5)
    validation_scores = []
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    for train_index, val_index in kf.split(X_train_scaled):
        # Split data into k-folds
        X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_kf, y_val_kf = y_train_scaled[train_index], y_train_scaled[val_index]

        # Create model using the provided parameters
        model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])

        # Optionally add learning rate scheduler
        optimizer = Adam(learning_rate=params['learning_rate'])
        model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"],)
        # Fit the model
        model.fit(X_train_kf, y_train_kf, epochs=100, validation_data=(X_val_kf, y_val_kf), callbacks=[early_stop], verbose=0)

        # Evaluate the model on the validation fold
        val_predictions = model.predict(X_val_kf)
        val_score = mean_squared_error(y_val_kf, val_predictions)
        validation_scores.append(val_score)

        # Clear the Keras session to prevent model/layer duplication
        K.clear_session()

    # Compute the average MSE over all folds
    average_score = np.mean(validation_scores)
    return {'loss': average_score, 'status': STATUS_OK}

def optimize(hidden_layers, trial):
    space = {
        'hidden_layers': hidden_layers,
        'units_1': hp.uniformint('units_1', 32, 256),
        'units_2': hp.uniformint('units_2', 32, 256) if hidden_layers > 1 else None,
        'units_3': hp.uniformint('units_3', 32, 256) if hidden_layers > 2 else None,
        'batch_normalization': hp.choice('batch_normalization', [False, True]),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 0.01),
        #'lr_schedule': hp.choice('lr_schedule', ['constant', 'linear', 'sinusoidal']),
        #'start_lr': hp.uniform('start_lr', 0.0001, 0.01),
        #'end_lr': hp.uniform('end_lr', 0.0001, 0.01),
        #'base_lr': hp.uniform('base_lr', 0.0001, 0.01),
        #'max_lr': hp.uniform('max_lr', 0.0001, 0.01),
    }
    best = fmin(
        fn=objective_nn,
        space=space,
        algo=tpe.suggest,
        trials=trial,
        max_evals=50,
        rstate=np.random.RandomState(seed)
    )
    # Check if the time limit flag was set to True
    if time_limit_reached:
        return best
        
    return best

In [28]:
global start_time, time_limit_reached

trial_nn2 = Trials()
time_limit_reached = False
best_hyperparams_2_layer = {}

best_hyperparams_2_layer[mode] = optimize(2, trial_nn2)
best_hyperparams_2_layer = convert_dict(best_hyperparams_2_layer)[mode]
print("Best Hyperparameters for 2 hidden layers:", best_hyperparams_2_layer)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 2ms/step 

23/23 [==============================] - 0s 2ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step         

In [29]:
# Train nn2_optimized with optimized hyperparameters for 2 hidden layers
nn2_optimized = create_model(2, [best_hyperparams_2_layer['units_1'], best_hyperparams_2_layer['units_2']], best_hyperparams_2_layer['batch_normalization'])
nn2_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_2_layer['learning_rate']))
early_stop = EarlyStopping(monitor='val_loss', patience=5)
nn2_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# if best_hyperparams_2_layer['lr_schedule'] == 0:# 'constant':
#     nn2_optimized = create_model(2, [best_hyperparams_2_layer['units_1'], best_hyperparams_2_layer['units_2']], best_hyperparams_2_layer['batch_normalization'])
#     nn2_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_2_layer['learning_rate']))
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn2_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# elif best_hyperparams_2_layer['lr_schedule'] == 1: #'linear':
#     nn2_optimized = create_model(2, [best_hyperparams_2_layer['units_1'], best_hyperparams_2_layer['units_2']], best_hyperparams_2_layer['batch_normalization'])
#     lr_scheduler = LinearLearningRateScheduler(best_hyperparams_2_layer['start_lr'], best_hyperparams_2_layer['end_lr'], 100)
#     nn2_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn2_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

# elif best_hyperparams_2_layer['lr_schedule'] == 2: #'sinusoidal':
#     nn2_optimized = create_model(2, [best_hyperparams_2_layer['units_1'], best_hyperparams_2_layer['units_2']], best_hyperparams_2_layer['batch_normalization'])
#     lr_scheduler = SinusoidalLearningRateScheduler(best_hyperparams_2_layer['base_lr'], best_hyperparams_2_layer['max_lr'], 100)
#     nn2_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn2_optimized.fit(X_train_scaled, y_train_scaled, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

nn2_optimized_pred = pd.DataFrame(nn2_optimized.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn2_optimized_pred_unscaled = target_scaler.inverse_transform(nn2_optimized_pred)
nn2_optimized_mse = mean_squared_error(y_test, nn2_optimized_pred_unscaled)
nn2_optimized_r2 = r2_score(y_test, nn2_optimized_pred_unscaled)
print("Neural Network 2 hidden layers Optimized MSE:", nn2_optimized_mse, "Neural Network 2 hidden layers Optimized R2:", nn2_optimized_r2)

29/29 [==============================] - 0s 1ms/step
Neural Network 2 hidden layers Optimized MSE: 4.184514097334525e-05 Neural Network 2 hidden layers Optimized R2: 0.7913407695147708


In [30]:
from src.models.learning_rate import *
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from tensorflow.keras import backend as K

def create_model(hidden_layers, units, batch_normalization):
    model = Sequential()
    model.add(Dense(units[0], input_dim=X_train_scaled.shape[1], activation='relu'))
    if batch_normalization:
        model.add(BatchNormalization())

    for i in range(1, hidden_layers):
        model.add(Dense(units[i], activation='relu'))
        if batch_normalization:
            model.add(BatchNormalization())

    model.add(Dense(1, activation='linear'))  # Assuming a regression problem
    return model

time_limit = 2*60*60
start_time = time.time()
time_limit_reached = False

def objective_nn(params):
    global start_time, time_limit_reached
    if (time.time() - start_time) > time_limit:
        time_limit_reached = True
        # Assign a large loss value to indicate failure
        return {'loss': float('inf'), 'status': STATUS_FAIL}

    # rest of your code
    model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])
    # Learning rate schedulers
    # if params['lr_schedule'] == 'linear':
    #     lr_scheduler = LinearLearningRateScheduler(params['start_lr'], params['end_lr'], 100)
    # elif params['lr_schedule'] == 'sinusoidal':
    #     lr_scheduler = SinusoidalLearningRateScheduler(params['base_lr'], params['max_lr'], 100)
        
    

    kf = KFold(n_splits=5)
    validation_scores = []
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    for train_index, val_index in kf.split(X_train_scaled):
        # Split data into k-folds
        X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_kf, y_val_kf = y_train_scaled[train_index], y_train_scaled[val_index]

        # Create model using the provided parameters
        model = create_model(params['hidden_layers'], [params['units_1'], params['units_2'], params['units_3']], params['batch_normalization'])

        # Optionally add learning rate scheduler
        optimizer = Adam(learning_rate=params['learning_rate'])
        model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"],)
        # Fit the model
        model.fit(X_train_kf, y_train_kf, epochs=100, validation_data=(X_val_kf, y_val_kf), callbacks=[early_stop], verbose=0)

        # Evaluate the model on the validation fold
        val_predictions = model.predict(X_val_kf)
        val_score = mean_squared_error(y_val_kf, val_predictions)
        validation_scores.append(val_score)

        # Clear the Keras session to prevent model/layer duplication
        K.clear_session()

    # Compute the average MSE over all folds
    average_score = np.mean(validation_scores)
    return {'loss': average_score, 'status': STATUS_OK}

def optimize(hidden_layers, trial):
    space = {
        'hidden_layers': hidden_layers,
        'units_1': hp.uniformint('units_1', 32, 256),
        'units_2': hp.uniformint('units_2', 32, 256) if hidden_layers > 1 else None,
        'units_3': hp.uniformint('units_3', 32, 256) if hidden_layers > 2 else None,
        'batch_normalization': hp.choice('batch_normalization', [False, True]),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 0.01),
        #'lr_schedule': hp.choice('lr_schedule', ['constant', 'linear', 'sinusoidal']),
        #'start_lr': hp.uniform('start_lr', 0.0001, 0.01),
        #'end_lr': hp.uniform('end_lr', 0.0001, 0.01),
        #'base_lr': hp.uniform('base_lr', 0.0001, 0.01),
        #'max_lr': hp.uniform('max_lr', 0.0001, 0.01),
    }
    best = fmin(
        fn=objective_nn,
        space=space,
        algo=tpe.suggest,
        trials=trial,
        max_evals=50,
        rstate=np.random.RandomState(seed)
    )
    # Check if the time limit flag was set to True
    if time_limit_reached:
        return best
        
    return best

In [31]:
global start_time, time_limit_reached

trial_nn2 = Trials()
time_limit_reached = False

trial_nn3 = Trials()
time_limit_reached = False
best_hyperparams_3_layer = {}
best_hyperparams_3_layer[mode] = optimize(3, trial_nn3)
best_hyperparams_3_layer = convert_dict(best_hyperparams_3_layer)[mode]
print("Best Hyperparameters for 3 hidden layers:", best_hyperparams_3_layer)

23/23 [==============================] - 0s 2ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step 

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 1ms/step                             

23/23 [==============================] - 0s 2ms/step                             

23/23 [==============================] - 0s 1ms/step         

In [32]:
# Train nn3_optimized with optimized hyperparameters for 3 hidden layers
nn3_optimized = create_model(3, [best_hyperparams_3_layer['units_1'], best_hyperparams_3_layer['units_2'], best_hyperparams_3_layer['units_3']], best_hyperparams_3_layer['batch_normalization'])
nn3_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_3_layer['learning_rate']))
early_stop = EarlyStopping(monitor='val_loss', patience=5)
nn3_optimized.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# if best_hyperparams_3_layer['lr_schedule'] == 0:#'constant':
#     nn3_optimized = create_model(3, [best_hyperparams_3_layer['units_1'], best_hyperparams_3_layer['units_2'], best_hyperparams_3_layer['units_3']], best_hyperparams_3_layer['batch_normalization'])
#     nn3_optimized.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=best_hyperparams_3_layer['learning_rate']))
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn3_optimized.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=0)

# elif best_hyperparams_3_layer['lr_schedule'] == 1: #'linear':
#     nn3_optimized = create_model(3, [best_hyperparams_3_layer['units_1'], best_hyperparams_3_layer['units_2'], best_hyperparams_3_layer['units_3']], best_hyperparams_3_layer['batch_normalization'])
#     lr_scheduler = LinearLearningRateScheduler(best_hyperparams_3_layer['start_lr'], best_hyperparams_3_layer['end_lr'], 100)
#     nn3_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn3_optimized.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

# elif best_hyperparams_3_layer['lr_schedule'] == 2: #'sinusoidal':
#     nn3_optimized = create_model(3, [best_hyperparams_3_layer['units_1'], best_hyperparams_3_layer['units_2'], best_hyperparams_3_layer['units_3']], best_hyperparams_3_layer['batch_normalization'])
#     lr_scheduler = SinusoidalLearningRateScheduler(best_hyperparams_3_layer['base_lr'], best_hyperparams_3_layer['max_lr'], 100)
#     nn3_optimized.compile(loss='mean_squared_error', optimizer=Adam())
#     early_stop = EarlyStopping(monitor='val_loss', patience=5)
#     nn3_optimized.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop, lr_scheduler], verbose=0)

nn3_optimized_pred = pd.DataFrame(nn3_optimized.predict(X_test_scaled), index=y_test.index, columns=['predictions'])
nn3_optimized_pred_unscaled = target_scaler.inverse_transform(nn3_optimized_pred)
nn3_optimized_mse = mean_squared_error(y_test, nn3_optimized_pred_unscaled)
nn3_optimized_r2 = r2_score(y_test, nn3_optimized_pred_unscaled)
print("Neural Network 3 hidden layers Optimized MSE:", nn3_optimized_mse, "Neural Network 3 hidden layers Optimized R2:", nn3_optimized_r2)


29/29 [==============================] - 0s 1ms/step
Neural Network 3 hidden layers Optimized MSE: 0.002586969328086822 Neural Network 3 hidden layers Optimized R2: -11.899825803701516


In [33]:
# write optimal hyperparameters to a csv file
nn1_optimal_params = pd.DataFrame(best_hyperparams_1_layer, index=['optimal_params']).T
nn1_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'nn1_optimal_params.csv'))
nn2_optimal_params = pd.DataFrame(best_hyperparams_2_layer, index=['optimal_params']).T
nn2_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'nn2_optimal_params.csv'))
nn3_optimal_params = pd.DataFrame(best_hyperparams_3_layer, index=['optimal_params']).T
nn3_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'nn3_optimal_params.csv'))

In [34]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import time

# Initialize global variable to track time
start_time = time.time()
time_limit_reached = False

def objective_rf(space):
    global start_time, time_limit_reached
    if (time.time() - start_time) > time_limit:
        time_limit_reached = True
        # Assign a large loss value to indicate failure
        return {'loss': float('inf'), 'status': STATUS_FAIL}

    
    model = RandomForestRegressor(
        n_estimators=int(space['n_estimators']),
        max_depth=int(space['max_depth']),
        min_samples_split=int(space['min_samples_split']),
        min_samples_leaf=int(space['min_samples_leaf']),
    )

    #score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    model.fit(X_train_, y_train_)
    score = mean_squared_error(model.predict(X_val), y_val)
    return {'loss': score, 'status': STATUS_OK}

def optimize_rf(trial):
    space = {
        'n_estimators': hp.uniformint('n_estimators', 10, 1000),
        'max_depth': hp.uniformint('max_depth', 2, 16),
        'min_samples_split': hp.uniformint('min_samples_split', 2, 10),
        'min_samples_leaf': hp.uniformint('min_samples_leaf', 1, 5),
    }
    best=None
    best = fmin(
            fn=objective_rf,
            space=space,
            algo=tpe.suggest,
            trials=trial,
            max_evals=50,  # Adjust as needed
            rstate=np.random.RandomState(seed)
        )
    # Check if the time limit flag was set to True
    if time_limit_reached:
        print("Optimization stopped due to time limit")
        return best
        
    return best

trial_rf = Trials()
RF_optimizations = {}
RF_optimizations[mode] = optimize_rf(trial_rf)

100%|██████████| 50/50 [20:41<00:00, 24.83s/trial, best loss: 4.0442982345918264e-05]


In [35]:
# Retrieve the best hyperparameters for RandomForestRegressor
RF_optimize = RF_optimizations[mode]

# Create and train the RandomForestRegressor with the optimized hyperparameters
regr_rf_optimized = RandomForestRegressor(
    n_estimators=int(RF_optimize['n_estimators']),
    max_depth=int(RF_optimize['max_depth']),
    min_samples_split=int(RF_optimize['min_samples_split']),
    min_samples_leaf=int(RF_optimize['min_samples_leaf']),
)
regr_rf_optimized.fit(X_train, y_train)

# Predict on the test set
regr_rf_optimized_pred = regr_rf_optimized.predict(X_test)

# Calculate Mean Squared Error and R-squared
regr_rf_optimized_mse = mean_squared_error(y_test, regr_rf_optimized_pred)
regr_rf_optimized_r2 = r2_score(y_test, regr_rf_optimized_pred)

# Print the scores
print("RandomForestRegressor Optimized MSE:", regr_rf_optimized_mse)
print("RandomForestRegressor Optimized R2:", regr_rf_optimized_r2)

RandomForestRegressor Optimized MSE: 3.6957701838024253e-05
RandomForestRegressor Optimized R2: 0.8157118019763194


In [36]:
# write optimal hyperparameters to a csv file
rf_optimal_params = pd.DataFrame(RF_optimizations[mode], index=['optimal_params']).T
rf_optimal_params.to_csv(os.path.join(models_folder, loc, turbine, 'hyperparameters', mode, 'rf_optimal_params.csv'))
rf_optimal_params

,optimal_params
max_depth,13.0
min_samples_leaf,4.0
min_samples_split,3.0
n_estimators,472.0
